In [33]:
import torch
from torch import nn
from d2l import torch as d2l

# 1. 基本操作

In [34]:
"""
 X: 输入张量
 K: 卷积核
"""
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

In [35]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
print(X)
print(K)
trans_conv(X, K)

tensor([[0., 1.],
        [2., 3.]])
tensor([[0., 1.],
        [2., 3.]])


tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [36]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
print(X)
print(K)

## 定义一个转置卷积层，输入和输出通道数为 1，卷积核大小为 2x2
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 1.],
          [2., 3.]]]])
tensor([[[[0., 1.],
          [2., 3.]]]])


tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [37]:
# 定义一个转置卷积层，输入和输出通道数为 1，卷积核大小为 2x2，padding=1
# 当将高和宽两侧的填充数指定为1时，转置卷积的输出中将删除第一和最后的行与列
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

步幅被指定为中间结果（输出），而不是输入
![转置卷积_步幅为2](./attach/转置卷积_步幅为2.png)

In [38]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [39]:
"""
先卷积再转置卷积
"""
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

# 3. 与矩阵变换的联系

In [40]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [41]:
# K是卷积核，把卷积核转成一个向量W（不用太在意为什么按照这种方式转）
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [42]:
"""
    1. 把X转成向量
    2. W与X相乘
    3. 得到的结果reshape(2, 2)

    和通过卷积得到的Y相同
"""
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [43]:
Z = trans_conv(Y, K)

"""
    1. 把Y转成向量
    2. W的转置与Y相乘
    3. 得到的结果reshape(3, 3)

    和通过转置卷积得到的Z相同
"""
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [44]:
# 以上也就说明了，为什么反向的卷积称为转置卷积的原因